In [3]:
import trainer.glue_base as glue_base
import models.sparse_token as sparse
import pickle, importlib
importlib.reload(glue_base)
importlib.reload(sparse)
Glue = glue_base.GlueAttentionApproxTrainer

In [4]:
subsets = ["cola","mnli","mrpc","qnli","qqp","rte","sst2","stsb","wnli",]
kss = [
    0.1, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.999, 'dynamic', 
    'dynamic:avg:avg:true', 'dynamic:avg:avg:false', 'dynamic:avg:max:true', 'dynamic:avg:max:false',
    'dynamic:max:avg:true', 'dynamic:max:avg:false', 'dynamic:max:max:true', 'dynamic:max:max:false',
]
sparse.benchmark_reset()
# subsets = ["mrpc"]
# kss = ['dynamic:avg:avg:f',0.1]

def get_score(score):
    if 'accuracy' in score:
        return score['accuracy'], "acc"
    first_metric = list(score.keys())[0]
    return score[first_metric], first_metric

results = {}
i = 0
for subset in subsets:
    trainer = Glue(dataset=subset, factor=16, batch_size=-1, device=0)
    trainer.load()
    scores = {}
    metric_name = ""
    bert_score, metric_name = get_score(trainer.eval_base_model())
    scores['bert'] = f'{bert_score:.5f}'
    for ks in kss:
        sparse.benchmark_reset()
        sparse_score, _ = get_score(trainer.eval_sparse_model(ks=ks))
        if isinstace(ks, str) and ks.startswith('dynamic'):
            est_k = sparse.benchmark_get_average('est_k')
            scores[str(ks)] = f'{sparse_score:.5f} (k:{est_k:.2f})'
        else:
            scores[str(ks)] = f'{sparse_score:.5f}'
        i += 1
        count = len(subsets) * len(kss)
        print(f'{i}/{count}')
    results[f"{subset} ({metric_name})"] = scores

with open('glue_benchmark.pkl', 'wb') as f:
    pickle.dump(results, f)

sparse.benchmark_report()

Trainer: mrpc


Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-7ab73b41110526b8.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-77eb4a03ea1c3167.arrow
Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-c3b7ac3527887b23.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9d

metric score {'accuracy': 0.8440579710144928, 'f1': 0.8865457612821593}
avg occupy 0.5399536083129833


eval:   0%|          | 0/216 [00:00<?, ?it/s]

avg avg False
avg avg False


eval:   0%|          | 0/216 [00:43<?, ?it/s]


KeyboardInterrupt: Interrupted by user

In [3]:
import pickle
import pandas as pd

with open('glue_benchmark.pkl', 'rb') as f:
    results = pickle.load(f)

data = []
subsets = list(results.keys())
factors = list(results[subsets[0]].keys())
for factor in factors:
    row = []
    for subset in subsets:
        row.append(results[subset][factor])
    data.append(row)
pd.DataFrame(data, columns=subsets, index=factors)

,cola (matthews_correlation),mnli (acc),mrpc (acc),qnli (acc),qqp (acc),rte (acc),sst2 (acc),stsb (pearson),wnli (acc)
bert,0.53388,0.84208,0.84406,0.91543,0.90908,0.72563,0.92431,0.88046,0.56338
0.1,0.00800,0.59042,0.33507,0.59967,0.72602,0.49819,0.69381,0.18832,0.60563
0.25,0.32926,0.81518,0.35768,0.84185,0.89043,0.68592,0.83486,0.62689,0.56338
0.375,0.45462,0.83780,0.50899,0.90079,0.90628,0.71841,0.88073,0.77045,0.56338
0.5,0.48454,0.84137,0.70957,0.91232,0.90878,0.73285,0.90367,0.84864,0.56338
0.625,0.50845,0.84208,0.81565,0.91470,0.90910,0.72202,0.92087,0.87315,0.56338
0.75,0.52069,0.84218,0.84058,0.91543,0.90910,0.72563,0.92317,0.87936,0.56338
0.875,0.53388,0.84218,0.84406,0.91543,0.90905,0.72563,0.92317,0.88040,0.56338
0.999,0.53388,0.84208,0.84406,0.91543,0.90908,0.72563,0.92431,0.88046,0.56338
dynamic,0.53388 (k:0.48),0.84004 (k:0.33),0.83942 (k:0.71),0.91287 (k:0.44),0.90920 (k:0.45),0.72563 (k:0.55),0.92317 (k:0.73),0.88043 (k:0.51),0.56338 (k:0.63)


In [ ]:
trainer = Glue(dataset='qnli', factor=16, batch_size=-1, device=0)
trainer.load() 

Trainer: qnli


Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-95cdbfccba10767f.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-a121d057d8b0504c.arrow
Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-81af1b1d0da19e67.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9d

In [5]:
trainer.model.bert = trainer.model_bert
trainer.eval_base_model()

eval: 100%|██████████| 683/683 [00:25<00:00, 26.83it/s]

metric score {'accuracy': 0.9154310818231741}
avg occupy 0.30243502260079336


{'accuracy': 0.9154310818231741}

In [9]:
trainer.seed()
import models.sparse_token as sparse
import transformers.models.bert.modeling_bert as berts
import importlib
importlib.reload(sparse)

wrapped_bert = sparse.ApproxSparseBertModel(trainer.model_bert, approx_bert=trainer.approx_bert, ks=0.1)
sparse_cls_bert = berts.BertForSequenceClassification(trainer.model_bert.config)
sparse_cls_bert.load_state_dict(trainer.model.state_dict())
sparse_cls_bert.bert = wrapped_bert
sparse_cls_bert.to(trainer.device).eval()

trainer.eval_base_model(model = sparse_cls_bert)

eval: 100%|██████████| 683/683 [00:36<00:00, 18.53it/s]


metric score {'accuracy': 0.5996705107084019}
avg occupy 0.30243502260079336


{'accuracy': 0.5996705107084019}